In [1]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.distributions import Normal
from torch.distributions.kl import kl_divergence

import os
import random

In [10]:
batch_size=256
epochs=500
seed=1
cuda=False and torch.cuda.is_available()
log_interval=1
r_dim=512
z_dim=512
result_path="results_np_z_y_hat_parmu_rdim+/"

In [3]:

torch.manual_seed(seed)
random.seed(seed)
device = torch.device("cpu") #"cuda" if args.cuda else 

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=False,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)


In [4]:
def get_context_idx(N):
    # generate the indeces of the N context points in a flattened image
    idx = random.sample(range(0, 784), N)
    idx = torch.tensor(idx, device=device)
    return idx
def generate_grid(h, w):
    rows = torch.linspace(0, 1, h, device=device)
    cols = torch.linspace(0, 1, w, device=device)
    grid = torch.stack([cols.repeat(h, 1).t().contiguous().view(-1), rows.repeat(w)], dim=1)
    grid = grid.unsqueeze(0)
    return grid
def idx_to_y(idx, data):
    # get the [0;1] pixel intensity at each index
    y = torch.index_select(data, dim=1, index=idx)
    return y
def idx_to_x(idx, batch_size):
    # From flat idx to 2d coordinates of the 28x28 grid. E.g. 35 -> (1, 7)
    # Equivalent to np.unravel_index()
    x = torch.index_select(x_grid, dim=1, index=idx)
    x = x.expand(batch_size, -1, -1)
    return x

In [5]:
class NP(nn.Module):
    def __init__(self, r_dim,z_dim):
        super(NP, self).__init__()
        self.r_dim = r_dim
        self.z_dim = z_dim
    
        self.h_1 = nn.Linear(3, 400)
        self.h_2 = nn.Linear(400, 400)
        self.h_3 = nn.Linear(400, self.r_dim)

        self.r_to_z_mean = nn.Linear(self.r_dim, self.z_dim)
        self.r_to_z_logvar = nn.Linear(self.r_dim, self.z_dim)

        self.g_1 = nn.Linear(self.z_dim + 2, 400)
        self.g_2 = nn.Linear(400, 400)
        self.g_3 = nn.Linear(400, 400)
        self.g_4 = nn.Linear(400, 400)
        self.g_y_mu = nn.Linear(400, 1)
        self.g_y_sigma = nn.Linear(400, 1)
        

    def h(self, x_y):
        x_y = F.relu(self.h_1(x_y))
        x_y = F.relu(self.h_2(x_y))
        x_y = F.relu(self.h_3(x_y))
        return x_y

    def aggregate(self, r):
        return torch.mean(r, dim=1)

    def g(self,z_sample, x_target):
        z_et_x = torch.cat([z_sample, x_target], dim=2)
        input = F.relu(self.g_1(z_et_x))
        input = F.relu(self.g_2(input))
        input = F.relu(self.g_3(input))
        input = F.relu(self.g_4(input))
        y_mu=self.g_y_mu(input)
        y_sigma=0.1 + 0.9 * F.softplus(self.g_y_sigma(input))
        return y_mu,y_sigma
    
    
    def xy_to_z_params(self, x, y):
        
        x_y = torch.cat([x, y], dim=2)
        
        r_i = self.h(x_y)
        r = self.aggregate(r_i)
        mu = self.r_to_z_mean(r)
        logvar = self.r_to_z_logvar(r)
        sigma=0.1+0.9*torch.sigmoid(logvar)
        return mu, sigma

    def forward(self, x_context, y_context, x_all=None, y_all=None):
        
        #produire z
        z_context_mu,z_context_sigma = self.xy_to_z_params(x_context, y_context)  # (mu, logvar) of z
        q_context = Normal(z_context_mu, z_context_sigma)
        # reconstruct the whole image including the provided context points
        x_target = x_grid.expand(y_context.shape[0], -1, -1)
        
        if self.training:  # loss function will try to keep z_context close to z_all         
            z_target_mu,z_target_sigma = self.xy_to_z_params(x_context, y_context)
            q_target = Normal(z_target_mu, z_target_sigma) 
            z_sample = q_target.rsample()
            
            z_sample = z_sample.unsqueeze(1).expand(-1, 784, -1)
            
            # Get parameters of output distribution
            y_pred_mu, y_pred_sigma = self.g(z_sample,x_target)
            p_y_pred = Normal(y_pred_mu, y_pred_sigma)

            return p_y_pred, q_target, q_context
        else:  # at test time we don't have the image so we use only the context
            z_sample = q_context.rsample()
            z_sample = z_sample.unsqueeze(1).expand(-1, 784, -1)
            # Predict target points based on context
            y_pred_mu, y_pred_sigma = self.g(z_sample,x_target)
            p_y_pred = Normal(y_pred_mu, y_pred_sigma)
            return p_y_pred,q_context,q_context

In [6]:

def np_loss(p_y_pred, y_target, q_target, q_context):
    
    #return logprob + KLD
    log_likelihood = p_y_pred.log_prob(y_target).mean(dim=0).sum()
    # KL has shape (batch_size, r_dim). Take mean over batch and sum over
    # r_dim (since r_dim is dimension of normal distribution)
    kl = kl_divergence(q_target, q_context).mean(dim=0).sum()
    return -log_likelihood + kl
    
    

In [7]:
model = NP(r_dim,z_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
x_grid = generate_grid(28, 28)
os.makedirs(result_path, exist_ok=True)

In [8]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (y_all, _) in enumerate(train_loader):
        
        batch_size = y_all.shape[0]
        
        y_all = y_all.to(device).view(batch_size, -1, 1)
        
        N = random.randint(1, 784)  # number of context points
        context_idx = get_context_idx(N)
        x_context = idx_to_x(context_idx, batch_size)
        y_context = idx_to_y(context_idx, y_all)
        x_all = x_grid.expand(batch_size, -1, -1)

        optimizer.zero_grad()
        
        p_y_pred, q_target, q_context = model(x_context, y_context, x_all, y_all)
        
        loss = np_loss(p_y_pred, y_all, q_target, q_context)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(y_all), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() / len(y_all)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))

In [9]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (y_all, _) in enumerate(test_loader):
            y_all = y_all.to(device).view(y_all.shape[0], -1, 1)
            batch_size = y_all.shape[0]

            N = 300
            context_idx = get_context_idx(N)
            x_context = idx_to_x(context_idx, batch_size)
            y_context = idx_to_y(context_idx, y_all)
            
            p_y_pred, q_target, q_context = model(x_context, y_context)
        
            
            test_loss += np_loss(p_y_pred, y_all, q_target, q_context).item()

            if i == 0:  # save PNG of reconstructed examples
                plot_Ns = [10, 100, 300, 784]
                num_examples = min(batch_size, 16)
                for N in plot_Ns:
                    recons = []
                    recons1=[]
                    recons2=[]
                    context_idx = get_context_idx(N)
                    x_context = idx_to_x(context_idx, batch_size)
                    y_context = idx_to_y(context_idx, y_all)
                    for d in range(5):
                        p_y_pred, _, _ = model(x_context, y_context)
                        
                        recons.append(p_y_pred.rsample()[:num_examples])
                        recons1.append(p_y_pred.mean[:num_examples])  
                        recons2.append(p_y_pred.stddev[:num_examples])  
                    recons = torch.cat(recons).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    recons1 = torch.cat(recons1).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    recons2 = torch.cat(recons2).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    background = torch.tensor([0., 0., 1.], device=device)
                    background = background.view(1, -1, 1).expand(num_examples, 3, 784).contiguous()
                    context_pixels = y_all[:num_examples].view(num_examples, 1, -1)[:, :, context_idx]
                    context_pixels = context_pixels.expand(num_examples, 3, -1)
                    background[:, :, context_idx] = context_pixels
                    comparison = torch.cat([background.view(-1, 3, 28, 28),
                                            recons,recons1,recons2])
                    save_image(comparison.cpu(),
                               result_path+"ep_" + str(epoch) +
                               "_nps_" + str(N) + ".png", nrow=num_examples)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.216817
Train Epoch: 1 [256/60000 (0%)]	Loss: 2.094654
Train Epoch: 1 [512/60000 (1%)]	Loss: 1.937303
Train Epoch: 1 [768/60000 (1%)]	Loss: 1.764454
Train Epoch: 1 [1024/60000 (2%)]	Loss: 1.543279
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.305979
Train Epoch: 1 [1536/60000 (3%)]	Loss: 1.071093
Train Epoch: 1 [1792/60000 (3%)]	Loss: 0.801306
Train Epoch: 1 [2048/60000 (3%)]	Loss: 1.073608
Train Epoch: 1 [2304/60000 (4%)]	Loss: 0.943537
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.740494
Train Epoch: 1 [2816/60000 (5%)]	Loss: 0.833270
Train Epoch: 1 [3072/60000 (5%)]	Loss: 0.888056
Train Epoch: 1 [3328/60000 (6%)]	Loss: 0.853924
Train Epoch: 1 [3584/60000 (6%)]	Loss: 0.830987
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.833829
Train Epoch: 1 [4096/60000 (7%)]	Loss: 0.828940
Train Epoch: 1 [4352/60000 (7%)]	Loss: 0.805490
Train Epoch: 1 [4608/60000 (8%)]	Loss: 0.785320
Train Epoch: 1 [4864/60000 (8%)]	Loss: 0.781485
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.77692

Train Epoch: 1 [42496/60000 (71%)]	Loss: 0.636258
Train Epoch: 1 [42752/60000 (71%)]	Loss: 0.686976
Train Epoch: 1 [43008/60000 (71%)]	Loss: 0.631447
Train Epoch: 1 [43264/60000 (72%)]	Loss: 0.725033
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.699216
Train Epoch: 1 [43776/60000 (73%)]	Loss: 0.676288
Train Epoch: 1 [44032/60000 (73%)]	Loss: 0.734521
Train Epoch: 1 [44288/60000 (74%)]	Loss: 0.729044
Train Epoch: 1 [44544/60000 (74%)]	Loss: 0.626967
Train Epoch: 1 [44800/60000 (74%)]	Loss: 0.656440
Train Epoch: 1 [45056/60000 (75%)]	Loss: 0.791782
Train Epoch: 1 [45312/60000 (75%)]	Loss: 0.713886
Train Epoch: 1 [45568/60000 (76%)]	Loss: 0.746380
Train Epoch: 1 [45824/60000 (76%)]	Loss: 0.760181
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.713750
Train Epoch: 1 [46336/60000 (77%)]	Loss: 0.690132
Train Epoch: 1 [46592/60000 (77%)]	Loss: 0.687306
Train Epoch: 1 [46848/60000 (78%)]	Loss: 0.669384
Train Epoch: 1 [47104/60000 (78%)]	Loss: 0.766302
Train Epoch: 1 [47360/60000 (79%)]	Loss: 0.742118


Train Epoch: 2 [24320/60000 (40%)]	Loss: 0.810103
Train Epoch: 2 [24576/60000 (41%)]	Loss: 0.766360
Train Epoch: 2 [24832/60000 (41%)]	Loss: 0.724283
Train Epoch: 2 [25088/60000 (42%)]	Loss: 0.720787
Train Epoch: 2 [25344/60000 (42%)]	Loss: 0.779110
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.749734
Train Epoch: 2 [25856/60000 (43%)]	Loss: 0.755822
Train Epoch: 2 [26112/60000 (43%)]	Loss: 0.793994
Train Epoch: 2 [26368/60000 (44%)]	Loss: 0.725668
Train Epoch: 2 [26624/60000 (44%)]	Loss: 0.706481
Train Epoch: 2 [26880/60000 (45%)]	Loss: 0.747830
Train Epoch: 2 [27136/60000 (45%)]	Loss: 0.698831
Train Epoch: 2 [27392/60000 (46%)]	Loss: 0.720145
Train Epoch: 2 [27648/60000 (46%)]	Loss: 0.765106
Train Epoch: 2 [27904/60000 (46%)]	Loss: 0.766286
Train Epoch: 2 [28160/60000 (47%)]	Loss: 0.754905
Train Epoch: 2 [28416/60000 (47%)]	Loss: 0.750329
Train Epoch: 2 [28672/60000 (48%)]	Loss: 0.757417
Train Epoch: 2 [28928/60000 (48%)]	Loss: 0.708186
Train Epoch: 2 [29184/60000 (49%)]	Loss: 0.773773


Train Epoch: 3 [6144/60000 (10%)]	Loss: 0.699464
Train Epoch: 3 [6400/60000 (11%)]	Loss: 0.693111
Train Epoch: 3 [6656/60000 (11%)]	Loss: 0.624933
Train Epoch: 3 [6912/60000 (11%)]	Loss: 0.703581
Train Epoch: 3 [7168/60000 (12%)]	Loss: 0.701532
Train Epoch: 3 [7424/60000 (12%)]	Loss: 0.645902
Train Epoch: 3 [7680/60000 (13%)]	Loss: 0.673999
Train Epoch: 3 [7936/60000 (13%)]	Loss: 0.618373
Train Epoch: 3 [8192/60000 (14%)]	Loss: 0.656266
Train Epoch: 3 [8448/60000 (14%)]	Loss: 0.653198
Train Epoch: 3 [8704/60000 (14%)]	Loss: 0.632027
Train Epoch: 3 [8960/60000 (15%)]	Loss: 0.644120
Train Epoch: 3 [9216/60000 (15%)]	Loss: 0.609017
Train Epoch: 3 [9472/60000 (16%)]	Loss: 0.609422
Train Epoch: 3 [9728/60000 (16%)]	Loss: 0.681602
Train Epoch: 3 [9984/60000 (17%)]	Loss: 0.654536
Train Epoch: 3 [10240/60000 (17%)]	Loss: 0.919281
Train Epoch: 3 [10496/60000 (17%)]	Loss: 0.652494
Train Epoch: 3 [10752/60000 (18%)]	Loss: 0.649261
Train Epoch: 3 [11008/60000 (18%)]	Loss: 0.704085
Train Epoch: 3 [

Train Epoch: 3 [48128/60000 (80%)]	Loss: -0.533040
Train Epoch: 3 [48384/60000 (80%)]	Loss: -0.693104
Train Epoch: 3 [48640/60000 (81%)]	Loss: -0.401304
Train Epoch: 3 [48896/60000 (81%)]	Loss: -0.633499
Train Epoch: 3 [49152/60000 (82%)]	Loss: -0.643000
Train Epoch: 3 [49408/60000 (82%)]	Loss: -0.701793
Train Epoch: 3 [49664/60000 (83%)]	Loss: 0.869437
Train Epoch: 3 [49920/60000 (83%)]	Loss: -0.318167
Train Epoch: 3 [50176/60000 (83%)]	Loss: -0.327317
Train Epoch: 3 [50432/60000 (84%)]	Loss: -0.026952
Train Epoch: 3 [50688/60000 (84%)]	Loss: -0.080359
Train Epoch: 3 [50944/60000 (85%)]	Loss: -0.159086
Train Epoch: 3 [51200/60000 (85%)]	Loss: -0.192440
Train Epoch: 3 [51456/60000 (86%)]	Loss: -0.344940
Train Epoch: 3 [51712/60000 (86%)]	Loss: -0.408389
Train Epoch: 3 [51968/60000 (86%)]	Loss: -0.577411
Train Epoch: 3 [52224/60000 (87%)]	Loss: -0.549854
Train Epoch: 3 [52480/60000 (87%)]	Loss: -0.378982
Train Epoch: 3 [52736/60000 (88%)]	Loss: -0.494713
Train Epoch: 3 [52992/60000 (88%

...

Train Epoch: 28 [47872/60000 (80%)]	Loss: -2.173857
Train Epoch: 28 [48128/60000 (80%)]	Loss: -1.953669
Train Epoch: 28 [48384/60000 (80%)]	Loss: -2.067007
Train Epoch: 28 [48640/60000 (81%)]	Loss: -2.127770
Train Epoch: 28 [48896/60000 (81%)]	Loss: -2.174577
Train Epoch: 28 [49152/60000 (82%)]	Loss: -2.238210
Train Epoch: 28 [49408/60000 (82%)]	Loss: -2.195006
Train Epoch: 28 [49664/60000 (83%)]	Loss: -2.217852
Train Epoch: 28 [49920/60000 (83%)]	Loss: -2.213627
Train Epoch: 28 [50176/60000 (83%)]	Loss: -2.265966
Train Epoch: 28 [50432/60000 (84%)]	Loss: -1.756701
Train Epoch: 28 [50688/60000 (84%)]	Loss: -2.142257
Train Epoch: 28 [50944/60000 (85%)]	Loss: -2.238319
Train Epoch: 28 [51200/60000 (85%)]	Loss: -2.218682
Train Epoch: 28 [51456/60000 (86%)]	Loss: -2.232857
Train Epoch: 28 [51712/60000 (86%)]	Loss: -2.208079
Train Epoch: 28 [51968/60000 (86%)]	Loss: -1.714774
Train Epoch: 28 [52224/60000 (87%)]	Loss: -2.119463
Train Epoch: 28 [52480/60000 (87%)]	Loss: -2.298487
Train Epoch:

Train Epoch: 29 [28160/60000 (47%)]	Loss: -2.139342
Train Epoch: 29 [28416/60000 (47%)]	Loss: -2.255322
Train Epoch: 29 [28672/60000 (48%)]	Loss: -2.313123
Train Epoch: 29 [28928/60000 (48%)]	Loss: -1.960490
Train Epoch: 29 [29184/60000 (49%)]	Loss: -2.210922
Train Epoch: 29 [29440/60000 (49%)]	Loss: -2.162041
Train Epoch: 29 [29696/60000 (49%)]	Loss: -2.278855
Train Epoch: 29 [29952/60000 (50%)]	Loss: -2.226190
Train Epoch: 29 [30208/60000 (50%)]	Loss: -2.269826
Train Epoch: 29 [30464/60000 (51%)]	Loss: -1.867841
Train Epoch: 29 [30720/60000 (51%)]	Loss: -2.129395
Train Epoch: 29 [30976/60000 (51%)]	Loss: -2.286480
Train Epoch: 29 [31232/60000 (52%)]	Loss: -2.254242
Train Epoch: 29 [31488/60000 (52%)]	Loss: -2.219635
Train Epoch: 29 [31744/60000 (53%)]	Loss: -2.196113
Train Epoch: 29 [32000/60000 (53%)]	Loss: -2.209433
Train Epoch: 29 [32256/60000 (54%)]	Loss: -2.270908
Train Epoch: 29 [32512/60000 (54%)]	Loss: -2.295125
Train Epoch: 29 [32768/60000 (54%)]	Loss: -2.201097
Train Epoch:

Train Epoch: 30 [8448/60000 (14%)]	Loss: -0.233941
Train Epoch: 30 [8704/60000 (14%)]	Loss: -2.213290
Train Epoch: 30 [8960/60000 (15%)]	Loss: -1.655079
Train Epoch: 30 [9216/60000 (15%)]	Loss: -2.201002
Train Epoch: 30 [9472/60000 (16%)]	Loss: -2.101868
Train Epoch: 30 [9728/60000 (16%)]	Loss: -2.131780
Train Epoch: 30 [9984/60000 (17%)]	Loss: -2.158323
Train Epoch: 30 [10240/60000 (17%)]	Loss: -2.209187
Train Epoch: 30 [10496/60000 (17%)]	Loss: -2.135284
Train Epoch: 30 [10752/60000 (18%)]	Loss: -1.348301
Train Epoch: 30 [11008/60000 (18%)]	Loss: -2.166722
Train Epoch: 30 [11264/60000 (19%)]	Loss: -2.181169
Train Epoch: 30 [11520/60000 (19%)]	Loss: -2.163856
Train Epoch: 30 [11776/60000 (20%)]	Loss: -2.227711
Train Epoch: 30 [12032/60000 (20%)]	Loss: -1.636066
Train Epoch: 30 [12288/60000 (20%)]	Loss: -2.172665
Train Epoch: 30 [12544/60000 (21%)]	Loss: -2.145412
Train Epoch: 30 [12800/60000 (21%)]	Loss: -1.866397
Train Epoch: 30 [13056/60000 (22%)]	Loss: -2.130183
Train Epoch: 30 [13

Train Epoch: 30 [48896/60000 (81%)]	Loss: -2.268771
Train Epoch: 30 [49152/60000 (82%)]	Loss: -2.124442
Train Epoch: 30 [49408/60000 (82%)]	Loss: -1.811578
Train Epoch: 30 [49664/60000 (83%)]	Loss: -2.117903
Train Epoch: 30 [49920/60000 (83%)]	Loss: -2.180516
Train Epoch: 30 [50176/60000 (83%)]	Loss: -2.223819
Train Epoch: 30 [50432/60000 (84%)]	Loss: -2.184037
Train Epoch: 30 [50688/60000 (84%)]	Loss: -1.573246
Train Epoch: 30 [50944/60000 (85%)]	Loss: -2.163821
Train Epoch: 30 [51200/60000 (85%)]	Loss: -1.285002
Train Epoch: 30 [51456/60000 (86%)]	Loss: -2.173414
Train Epoch: 30 [51712/60000 (86%)]	Loss: -2.236550
Train Epoch: 30 [51968/60000 (86%)]	Loss: -1.922216
Train Epoch: 30 [52224/60000 (87%)]	Loss: -2.113030
Train Epoch: 30 [52480/60000 (87%)]	Loss: -2.125519
Train Epoch: 30 [52736/60000 (88%)]	Loss: -2.223017
Train Epoch: 30 [52992/60000 (88%)]	Loss: -2.248879
Train Epoch: 30 [53248/60000 (89%)]	Loss: -2.288983
Train Epoch: 30 [53504/60000 (89%)]	Loss: -2.212318
Train Epoch:

Train Epoch: 31 [29184/60000 (49%)]	Loss: -2.125423
Train Epoch: 31 [29440/60000 (49%)]	Loss: -2.190852
Train Epoch: 31 [29696/60000 (49%)]	Loss: -2.213368
Train Epoch: 31 [29952/60000 (50%)]	Loss: -2.229831
Train Epoch: 31 [30208/60000 (50%)]	Loss: -2.280880
Train Epoch: 31 [30464/60000 (51%)]	Loss: -2.316806
Train Epoch: 31 [30720/60000 (51%)]	Loss: -2.316597
Train Epoch: 31 [30976/60000 (51%)]	Loss: -2.245446
Train Epoch: 31 [31232/60000 (52%)]	Loss: -2.348000
Train Epoch: 31 [31488/60000 (52%)]	Loss: -1.761893
Train Epoch: 31 [31744/60000 (53%)]	Loss: -2.107402
Train Epoch: 31 [32000/60000 (53%)]	Loss: -2.279256
Train Epoch: 31 [32256/60000 (54%)]	Loss: -2.275649
Train Epoch: 31 [32512/60000 (54%)]	Loss: -2.292860
Train Epoch: 31 [32768/60000 (54%)]	Loss: -2.399441
Train Epoch: 31 [33024/60000 (55%)]	Loss: -2.113889
Train Epoch: 31 [33280/60000 (55%)]	Loss: -2.088240
Train Epoch: 31 [33536/60000 (56%)]	Loss: -2.345133
Train Epoch: 31 [33792/60000 (56%)]	Loss: -2.347113
Train Epoch:

In [ ]:
torch.save(model.state_dict(), result_path+"model_{}.pt".format(r_dim))

In [70]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs=500

In [13]:

model.load_state_dict(torch.load(result_path+"model.pt"))
model.eval()

NP(
  (h_1): Linear(in_features=3, out_features=400, bias=True)
  (h_2): Linear(in_features=400, out_features=400, bias=True)
  (h_3): Linear(in_features=400, out_features=128, bias=True)
  (r_to_z_mean): Linear(in_features=128, out_features=128, bias=True)
  (r_to_z_logvar): Linear(in_features=128, out_features=128, bias=True)
  (g_1): Linear(in_features=130, out_features=400, bias=True)
  (g_2): Linear(in_features=400, out_features=400, bias=True)
  (g_3): Linear(in_features=400, out_features=400, bias=True)
  (g_4): Linear(in_features=400, out_features=400, bias=True)
  (g_y_mu): Linear(in_features=400, out_features=1, bias=True)
  (g_y_sigma): Linear(in_features=400, out_features=1, bias=True)
)